In [ ]:
import brightway2 as bw
import numpy as np
import pandas as pd
from copy import copy

#Local files
import json, os

In [ ]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']

# Setup the project

### TODO -> choose project

In [ ]:
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

# Import older versions of databases

### TODO -> give paths to databases files
for now let's use ecoinvent 3.3 cutoff and EXIOBASE 2.2

In [ ]:
if which_pc == 'local':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
elif which_pc == 'merlin':
    ex22_path = '/data/user/kim_a/LCA_files/exiobase_22/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    ei36_path = '/data/user/kim_a/LCA_files/ecoinvent_36_cutoff/datasets'  
    ag13_path = '/data/user/kim_a/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'

## 1. Biosphere3

In [ ]:
bw.bw2setup()

## 2. Ecoinvent 3.3

In [ ]:
ei33_name = "ecoinvent 3.3 cutoff"  
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33 = bw.SingleOutputEcospold2Importer(ei33_path, ei33_name)
    ei33.apply_strategies()
    ei33.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei33.statistics()

In [ ]:
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33.write_database()

## 3. Exiobase 2.2

In [ ]:
# 2. Exiobase 
from bw2io.importers.exiobase2 import Exiobase22Importer

In [ ]:
ex22_name = 'EXIOBASE 2.2'
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22 = Exiobase22Importer(ex22_path, ex22_name)
    ex22.apply_strategies()

In [ ]:
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22.write_database()

## 4. Agribalyse 1.2
better see: [how to import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%20with%20Ecoinvent%202.2.ipynb)

In [ ]:
def import_agribalyse12(ag12_path, ag12_name, ei_name):

    ag12_ei_name = ag12_name + ' - ' + ei_name
    
    if ei_name not in bw.databases:
        print('Cannot find database: ' + ei_name)
        return
    
    if ag12_ei_name in bw.databases:
        print(ag12_ei_name + " database already present!!! No import is needed")
    else:
        ag12_ei = bw.SingleOutputEcospold1Importer(ag12_path, ag12_ei_name)
        ag12_ei.apply_strategies()
        ag12_ei.statistics()
        
        ag12_ei.write_excel(True)
        from bw2io.strategies.simapro import normalize_simapro_biosphere_categories, normalize_simapro_biosphere_names
        ag12_ei.apply_strategy(normalize_simapro_biosphere_categories)
        ag12_ei.apply_strategy(normalize_simapro_biosphere_names)

        from bw2io.strategies import link_iterable_by_fields
        import functools
        ag12_ei.apply_strategy(functools.partial(link_iterable_by_fields, other=bw.Database("biosphere3"), kind="biosphere"))
        ag12_ei.statistics()

        ag12_ei.write_excel(True)
        
        ag12_ei_new_biosphere_name = ag12_ei_name + " - new biosphere"
        bw.Database(ag12_ei_new_biosphere_name).register()
        ag12_ei.add_unlinked_flows_to_biosphere_database(ag12_ei_new_biosphere_name)

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and exc['name'] == 'Alfalfa, conventional, for animal feeding, at farm gate':
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        from bw2io.strategies import link_technosphere_based_on_name_unit_location
        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ag12_ei.statistics()

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and not exc.get('input'):
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        for ds in ag12_ei.data:
            for exc in ds['exchanges']:
                if exc['type'] == 'production' and not exc.get('input'):
                    print("Fixing:", exc['name'])
                    exc['type'] = 'technosphere'

        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ei_name_short = ei_name #TODO
        ag12_ei.apply_strategy(functools.partial(link_technosphere_based_on_name_unit_location, external_db_name=ei_name_short))
        ag12_ei.statistics()

        ag12_ei.add_unlinked_activities()
        ag12_ei.statistics()

        for ds in ag12_ei.data:
            del ds['code']

        from bw2io.strategies import set_code_by_activity_hash
        ag12_ei.apply_strategy(set_code_by_activity_hash)

        ag12_ei.apply_strategy(functools.partial(
                link_iterable_by_fields,
                other=ag12_ei.data,
                fields=('name', 'location', 'unit'),
                relink=True
        ))
        ag12_ei.statistics()
        
        if ag12_ei_name in bw.databases:
            print(ag12_ei_name + " database already present!!! No import is needed")
        else:
            ag12_ei.write_database()

In [ ]:
ag12_name = 'Agribalyse 1.2'
import_agribalyse12(ag12_path, ag12_name, ei33_name)

In [ ]:
bw.databases

## 5. HEIA

### TODO -> ask Andi for files

# Import newer versions of databases

## 1. Ecoinvent 3.6 cutoff

In [ ]:
ei36_name = "ecoinvent 3.6 cutoff"
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36 = bw.SingleOutputEcospold2Importer(ei36_path, ei36_name)
    ei36.apply_strategies()
    ei36.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei36.statistics()

In [ ]:
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36.write_database()

In [ ]:
bw.databases

## 2. Agribalyse 1.3

In [ ]:
if 'Agribalyse 1.3 - ecoinvent 3.6 cutoff' in bw.databases:
    del bw.databases['Agribalyse 1.3 - ecoinvent 3.6 cutoff']

### TODO -> Chris, Rhythima, check if migrations are correct

In [ ]:
ei_name = ei36_name

In [ ]:
ag13_name = 'Agribalyse 1.3'
ag13_ei_name = ag13_name + ' - ' + ei_name
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    ag13_ei = bw.SimaProCSVImporter(ag13_path, ag13_ei_name)
    ag13_ei.apply_strategies()
    
    # Apply all migrations with previous versions of ecoinvent
    ag13_ei.migrate('simapro-ecoinvent-3.3')
    
    # Update US locations
    from bw2io.strategies.locations import update_ecoinvent_locations
    ag13_ei = update_ecoinvent_locations(ag13_ei)
    
    # Biosphere flows
    ag13_ei_new_biosphere_name = ag13_ei_name + " - new biosphere"
    bw.Database(ag13_ei_new_biosphere_name).register()
    ag13_ei.add_unlinked_flows_to_biosphere_database(ag13_ei_new_biosphere_name)
    
    # Add unlinked waste flows as new activities
    ag13_ei.add_unlinked_activities()
    
    ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
    ag13_ei.statistics()

### Define some of the migrations manually.
- Most of them are minor changes in names of activities and reference products
- Some activities contain `multiplier` field if the unit conversion is needed or reference products are not identical

In [ ]:
agribalyse13_ecoinvent36_names = {
    'fields': ['name'],
    'data': [
        (
            ['extrusion production, plastic film'],
            {
                'name': 'extrusion, plastic film'
            }
        ),
        (
            ['extrusion production, plastic pipes'],
            {
                'name': 'extrusion, plastic pipes'
            }
        ),
        (
            ['blow moulding production'],
            {
                'name': 'blow moulding'
            }
        ),
        (
            ['calendering production, rigid sheets'],
            {
                'name': 'calendering, rigid sheets'
            }
        ),
        (
            ['steam production in chemical industry'],
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # change of units, see comment on this activity in ecoinvent
                # TODO does this multiplier also change uncertainty distribution??
            }
        ),
        (
            ['water production, completely softened, from decarbonised water, at user'],
            {
                'name': 'water production, completely softened',
                'reference product': 'water, completely softened'
            }
        ),
        (
            ['market for water, decarbonised, at user'],
            {
                'name': 'market for water, decarbonised',
                'reference product': 'water, decarbonised'
            }
        ),
        (
            ['textile production, jute'],
            {
                'name': 'textile production, jute, weaving'
            }
        ),
        (
            ['market for transport, freight, sea, transoceanic ship'],
            {
                'name': 'market for transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['transport, freight, sea, transoceanic ship'],
            {
                'name': 'transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['oxidation of manganese dioxide'],
            {
                'reference product': 'potassium permanganate',
                'multiplier': 90/1000 # change of reference product, see comment on this activity in ecoinvent
            }
        ),
        (
            ['irrigation'],
            {
                'name': 'market for irrigation',
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-names").write(
    agribalyse13_ecoinvent36_names,
    description="Change names of some activities"
)

### Define migrations for market groups

In [ ]:
agribalyse13_ecoinvent36_market_groups = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['market for electricity, low voltage', 'BR'],
            {
                'name': 'market group for electricity, low voltage'
            }
        ),
        (
            ['market for electricity, medium voltage', 'BR'],
            {
                'name': 'market group for electricity, medium voltage'
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-market-groups").write(
    agribalyse13_ecoinvent36_market_groups,
    description="Change markets to market groups for Brazil"
)

In [ ]:
ag13_ei.migrate('agribalyse13-ecoinvent36-names')
ag13_ei.migrate('agribalyse13-ecoinvent36-market-groups')
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
ag13_ei.statistics()

### Allocate by production volume

In [ ]:
from utils import *

In [ ]:
ei36 = bw.Database(ei_name)

In [ ]:
# Create mapping between unlinked exchanges and ecoinvent activities that constitute each unlinked exchange.
# In this case we do not need to do it manually since all unlinked exchanges need to be allocated geographically.
# Example: (market for lime, GLO) is split by production volume into (market for lime, RoW) & (market for lime, RER).
# Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
# The key is the name of the unlinked exchange and the values are ecoinvent activities codes.

def create_location_mapping(ag13_ei, ei_name):
    
    ei36 = bw.Database(ei_name)
    
    unlinked_list = list(ag13_ei.unlinked)
    len_unlinked  = len(unlinked_list)

    mapping = [0]*len_unlinked

    for u in range(len_unlinked):
        new_el = {}
        name = unlinked_list[u]['name'] 
        loc  = unlinked_list[u]['location']
        acts_codes = [act['code'] for act in ei36 if name == act['name']]
        new_el[ (name, loc) ] = acts_codes
        mapping[u] = new_el
        
    return mapping

In [ ]:
mapping = create_location_mapping(ag13_ei, ei_name)

In [ ]:
agg = modify_exchanges(ag13_ei, mapping, ei_name)

In [ ]:
agg.statistics()

### Change uncertainty info


In [ ]:
import stats_arrays as sa

In [ ]:
changed = []
for i,act in enumerate(agg.data):
    excs = act.get('exchanges', [])
    for j,exc in enumerate(excs):
        if exc.get('uncertainty type', False) == sa.LognormalUncertainty.id and \
        np.allclose(exc.get('amount'), exc.get('loc')):
            exc.update(loc=np.log(exc['amount']))
            changed.append([i,j])

In [ ]:
assert len(changed)==319

### Write database

In [ ]:
# if ag13_ei_name in bw.databases:
#     print(ag13_ei_name + " database already present!!! No import is needed")
# else:
#     agg.write_database()

In [ ]:
a = 'ammonium nitrate phosphate production'
b = 'diammonium phosphate production'
c = 'monoammonium phosphate production'

In [ ]:
for act in agg.data:
    if act['name'] in [a,b,c]:
        print(act)

In [ ]:
# Remove repetitive activities
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'ammonium nitrate phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '(1,1,5,1,1,na); assimilation MAP', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': 'de221991cc69f37976042f05f448c94c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'diammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '64037e162f6d6d1048470c3a1135f4fb', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'monoammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '6d61eb45c1d285073770aa839426d97c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})


In [ ]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

In [ ]:
bw.databases

# Exiobase 3.3.7